# Study B
This notebook shows the full workflow for building models, simulating growth and obtaining SCFA predictions from data collected by the _ex vivo_ study conducted by the Hamaker Lab in 2021 (Study B)

In [ ]:
import pandas as pd
import micom
import micom.measures
from plotnine import *
import scipy

%matplotlib inline

## SCFA Flux
Here we will gather the net SCFA production across the culturing period. The "Measured" columns contain the measured flux over the course of the experiment (mM/h, [T1]-[T0]/h).

In [ ]:
scfa=pd.read_csv('../data/raw_data/studyB_original_scfas.csv', index_col = 0)

scfa['donor'] = scfa.index.str.split('_').str[0]
scfa['treatment'] = scfa.index.str.split('_').str[1].str.capitalize()
scfa['treatment'] = scfa['treatment'].str.replace('Fos','FOS')

scfa = scfa.groupby(['donor','treatment']).mean().reset_index()

scfa['sample_id'] = scfa['donor']+'_'+scfa['treatment']
scfa.set_index('sample_id', inplace = True)

scfa

## Taxonomy Table
Next, we will read in abundance data for all samples. We will convert this into a taxnomy table to use in MICOM

In [ ]:
abundance = pd.read_csv('../data/raw_data/studyB_original_abundance.csv', index_col = 0) # Load the abundance
abundance['donor'] = abundance['donor'].astype('str')
abundance['sample_id'] = abundance[['treatment_II_B','donor']].agg(
    '_'.join, axis=1) # Join the treatment and donor into sample ID
abundance = abundance.drop(columns = ['treatment_II_B','donor'])
abundance.set_index('sample_id',inplace = True)
abundance = abundance.sort_index() #sort the index in order
abundance = abundance[~abundance.index.str.contains('II_')] # Remove the treatment without SCFA measurements
abundance.reset_index(inplace = True)
vals = abundance.groupby(abundance['sample_id']).cumcount().values# add replicate number
abundance['sample_id'] = (abundance['sample_id']+'_'+ (vals+1).astype(str))
abundance = pd.melt(abundance,
                    id_vars = 'sample_id', 
                    value_vars = abundance.columns[1:], 
                    var_name = 'id', 
                    value_name = 'abundance') #melt into taxonomy table 
abundance['id'] = (abundance['id'].
                   str.replace('k__','').
                   str.replace('p__','').
                   str.replace('c__','').
                   str.replace('o__','').
                   str.replace('f__','').
                   str.replace('g__','').
                   str.replace('s__','')) # replace leading taxonomic identifiers
abundance['genus'] = abundance['id'].str.split(';').str[-2]
abundance = abundance[(abundance['sample_id'].str.contains('Blank'))|
                      (abundance['sample_id'].str.contains('Pectin'))|
                      (abundance['sample_id'].str.contains('FOS'))] # isolate treatment samples
abundance['id'] = abundance['genus']
abundance = abundance.groupby(['sample_id','id','genus']).sum().reset_index().dropna() # sum duplicates
table = pd.pivot_table(abundance, 
               index = 'sample_id',
               columns = 'genus',
               values = 'abundance').fillna(0.0) # build abundance matrix
abundance

## Build Models
Now, we'll build our models, with cutoff of 0.001

In [ ]:
manifest = micom.workflows.build(abundance,
                                 out_folder = '../models/studyB',
                                 model_db = '../agora103_genus.qza',
                                 cutoff = 0.001, 
                                 threads = 10)

## Load Medium
Load in the study-specific carbon-stripped European Diet with dilution-adjusted residual fiber, and construct the intervention diets by augmenting with FOS and pectin

In [ ]:
medium = pd.read_csv('../media/studyBmedium.csv')

controlMedium = medium

pectMedium = pd.concat([medium,
                           pd.DataFrame({'reaction':['EX_pect_m'],
                                         'flux':[1]
                                        })]) 
fosMedium = pd.concat([medium,
                           pd.DataFrame({'reaction':['EX_kestopt_m', 'EX_kestottr_m', 'EX_kesto_m'],
                                         'flux':[100,100,100]
                                        })])

## Grow Models
Now we'll grow the samples using the respective media we constructed

In [ ]:
manifest = pd.read_csv('../models/studyB/manifest.csv')
controlManifest = manifest[manifest.sample_id.str.startswith('Blank')]
pectManifest = manifest[manifest.sample_id.str.startswith('Pect')]
fosManifest = manifest[manifest.sample_id.str.startswith('FOS')]
controlGrowth = micom.workflows.grow(controlManifest,
                                     '../models/studyB/',
                                     medium=controlMedium,
                                     tradeoff= 0.7, 
                                     strategy='none',
                                     threads = 10)

pectGrowth = micom.workflows.grow(pectManifest,
                                  '../models/studyB/',
                                  medium = pectMedium, 
                                  tradeoff = 0.7,
                                  strategy = 'none',
                                  threads = 10)

fosGrowth = micom.workflows.grow(fosManifest,
                                     '../models/studyB/',
                                     medium=fosMedium,
                                     tradeoff=0.7, 
                                     strategy='none',
                                     threads = 10)

## Compare SCFA Fluxes
We can now extract the production fluxes of SCFAs from each growth simulation, and construct a dataframe with both measured and predicted production rates.

In [ ]:
controlProduction = micom.measures.production_rates(controlGrowth) # Calculate production rates
pectinProduction = micom.measures.production_rates(pectGrowth)
fosProduction = micom.measures.production_rates(fosGrowth)


production = pd.concat([controlProduction,
                        pectinProduction, 
                        fosProduction]) # concatenate results 
res = production[(production['name']==('butyrate'))|
                 (production['name']==('acetate'))|
                 (production['name']==('propionate'))] # filter to SCFA production 
res = pd.pivot(res, index = 'sample_id',
                    columns = 'name',
                    values = 'flux') # pivot data
res.rename(columns = {'acetate':'acetatePredicted',
                      'butyrate':'butyratePredicted',
                      'propionate':'propionatePredicted'
                      }, inplace = True # rename columns for clarity 
          )

res['donor'] = res.index.str.split('_').str[1]
res['treatment'] = res.index.str.split('_').str[0]
res = res.groupby(['donor','treatment']).mean().reset_index()
res['sample_id'] = res['donor']+'_'+res['treatment']
res.set_index('sample_id', inplace = True)
res = pd.concat([res, scfa], axis = 1)
res = res.loc[:,~res.columns.duplicated()].copy()
res['treatment'] = res['treatment'].str.replace('Blank','Control')
res

## Plot Results
Finally, we'll plot predicted vs measured fluxes against each other

In [ ]:
fig1 = (ggplot(
    res, aes(x = 'butyrateMeasured', y = 'butyratePredicted'))
    +geom_smooth(method = 'lm', linetype = '--')
    +geom_point(aes(color = 'treatment'), size = 8)
    +scale_color_manual(limits = ['Control','Pectin', 'FOS'], 
                        values = ['cornflowerblue', 'mediumseagreen', 'purple'])
    +labs(title='Butyrate',
          x='Measured Butyrate (mmol/L/h)',
          y = 'Predicted Butyrate (mmol/gDW/h)',
          color = 'Treatment')
    +theme(text = element_text(size=35, color = 'black'),panel_background=element_rect(fill = "white",
                                    colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                    axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                    legend_position='right',axis_text_x=element_text(rotation = 20, hjust = 1))
)
fig1

In [ ]:
scipy.stats.linregress(res['butyratePredicted'],
                       res['butyrateMeasured'])

In [ ]:
fig2 = (ggplot(
    res, aes(x = 'propionateMeasured', y = 'propionatePredicted'))
    +geom_smooth(method = 'lm', linetype = '--')
    +geom_point(aes(color = 'treatment'), size = 8)
    +scale_color_manual(limits = ['Control','Pectin', 'FOS'], 
                        values = ['cornflowerblue', 'mediumseagreen', 'purple'])
    +labs(title='Propionate',
          x='Measured Propionate (mmol/L/h)',
          y = 'Predicted Propionate (mmol/gDW/h)',
          color = 'Treatment')
    +theme(text = element_text(size=35, color = 'black'),panel_background=element_rect(fill = "white",
                                    colour = "white",size = 0.5, linetype = "solid"),panel_grid= element_blank(),
                                    axis_line = element_line(size = 2, linetype = "solid",colour = "black"),
                                    legend_position='right',axis_text_x=element_text(rotation = 20, hjust = 1))
)
fig2

In [ ]:
scipy.stats.linregress(res['propionatePredicted'],
                       res['propionateMeasured'])

## Save all results

In [ ]:
res.to_csv('../results/studyB.csv')